In [ ]:
import requests

API_URL = "https://www.worldcubeassociation.org/api/v0"
API_PARAMS = "&isActive=true&isHidden=false&per_page=100"

def parse_roles(data):
    roles = []
    for role in data:
        wca_id = role["user"]["wca_id"]
        group_name = role["group"]["name"]
        group_type = role["group"]["group_type"]

        if not wca_id or not group_type:
            continue

        roles.append((wca_id, group_name, group_type))

    # De-duplicate
    roles = list(set(roles))

    # Sort by group_type, then group_name, then wca_id
    roles = sorted(roles, key=lambda x: x[2] + x[1] + x[0])

    return roles

# This function is for group types that don't have subgroups
def get_roles(group_type):
    print(f"Fetching {group_type}")
    data = requests.get(f"{API_URL}/user_roles?groupType={group_type}{API_PARAMS}").json()
    return parse_roles(data)

# This function is for group types that have subgroups
def get_groups_then_roles(group_type):
    print(f"Fetching {group_type} groups")
    data = requests.get(f"{API_URL}/user_groups?groupType={group_type}{API_PARAMS}").json()
    group_ids = [group["id"] for group in data]

    roles = []

    for i, group_id in enumerate(group_ids):
        print(f"Group {group_id} ({i+1}/{len(group_ids)})")
        data = requests.get(f"{API_URL}/user_roles?groupId={group_id}{API_PARAMS}").json()
        roles.extend(parse_roles(data))
    
    return roles

In [ ]:
officers_roles = get_roles("officers")
board_roles = get_roles("board")
translators_roles = get_roles("translators")

delegates_roles = get_groups_then_roles("delegate_regions")
teams_committees_roles = get_groups_then_roles("teams_committees")

In [ ]:
import pandas as pd

# Put all roles in one dataframe
df = pd.DataFrame(
    officers_roles + board_roles + translators_roles + delegates_roles + teams_committees_roles,
    columns=["wca_id", "group_name", "group_type"],
)
df.to_csv("WCA_export/Staff.tsv", index=False, sep="\t")
df.head()